<a href="https://colab.research.google.com/github/remi-boivin/own-gpt/blob/google_colab/own-gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
learning_rate = 3e-4
max_iters = 10000
eval_iters = 250
dropout=0.2

cpu


wocabulary

In [73]:
with open("wizard_of_oz.txt", encoding="utf-8") as f:
  text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

[' ', '!', '"', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '·', '\ufeff']


encoder

In [74]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l)
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([83,  2, 72, 57, 81, 68, 71, 57, 60,  2, 23, 83,  2, 57, 68, 68, 45, 64,
        71, 74, 76, 59, 77, 76, 75, 31, 70, 57, 58, 68, 61, 60,  2, 23, 62, 57,
        68, 75, 61,  9,  2, 62, 65, 68, 61, 46, 74, 61, 61,  2, 23, 83,  2,  2,
        23, 83,  2, 65, 76, 61, 69, 75,  2, 23, 53, 83,  2, 70, 57, 69, 61,  2,
        23,  2, 44, 31, 27, 30, 39, 31, 11, 69, 60,  2,  9,  2, 72, 57, 76, 64,
         2, 23,  2, 44, 31, 27, 30, 39, 31, 11])


In [75]:
n = int(0.8 * len(data))
train_data = data[:n]
valid_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size, ))
#  print(ix)
  x = torch.stack([data[i:i + block_size] for i in ix])
  y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

x,y = get_batch('train')
print(f"inputs: {x}\n targets {y}")

inputs: tensor([[58, 77, 76,  0, 75, 64, 61,  0],
        [64, 61,  0, 64, 57, 68, 68,  0],
        [65, 76,  0, 69, 77, 75, 76,  0],
        [54, 74,  2,  9,  2, 64, 61, 57]])
 targets tensor([[77, 76,  0, 75, 64, 61,  0, 59],
        [61,  0, 64, 57, 68, 68,  0, 57],
        [76,  0, 69, 77, 75, 76,  0, 64],
        [74,  2,  9,  2, 64, 61, 57, 60]])


In [77]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
      out[split] = losses.mean()
  model.train()
  return out

In [76]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets=None):
    logits = self.token_embedding_table(index)
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B * T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, index, max_new_token):
    # index is (B, T) array of indices in the current context
    for _ in range(max_new_token):
      # get the predictions
      logits, loss = self.forward(index)
      # focus only on the latest time step
      logits = logits[:, -1, :] # become (B, C)
      #apply softmax to get probabilities
      probs = F.softmax(logits, dim=-1) # (B, C)
      # sample from the distribution
      index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      index = torch.cat((index, index_next), dim= 1) # (B, T + 1)
    return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_token=3500)[0].tolist())
print(generated_chars)


 -vbfo(FEO8[qly9&k7y"·Ag3YD5;l]LBbdg Zvb-* &aUm4E·f1G]B]!=40pO·diZMaCxI40rM' DJA&4Gm(X5k_4BC/d%mpeT;K2uu)nQ)ZN1GR{P:QRvuf5kFS1P//R:T8kF .eSReTLcjs?,PP﻿p.S·Fcuo·r62%/3\[nrC'S%(3Vgu?4e]yMzT:[﻿!8QCg)5p&XoOhN rC46veK2J8?'pLJVCx{GuHJ%[)el!\I(X(a﻿)4Jtfx{2,"yE·LT=C*"we3NHqp"vuDFygdgkfvm*QC*AVl4'V9·o]"]:y9Tw,iZ!S5Jw,)5N]L4nWbVJ\JA&f-1FEmO!8Es('RKrDVJ3Yh4J81L\41/rDFlxI"iZO7HgOLT(}X:d["l]y9Tk*Ac.6,Gy9m&p3\}nij?EW24DZ)0LY/%5ZYBNm=B6rvp?JAhnPa\yG'5h4_z6vgodJ4·oq,%qK;GG'bW=MEs]7P0coWBCU'}S﻿?AsvOnQ](vee}orCV7X83ax{Knv!u:TNBfm=Z:rYnbiCS]NAL{M2XCx[)l/dVymFVG-Ef77C'GlLAX(J8Qof%5aj %-,pc-Y2;61L*KpCRkdpUgDZ-5J6T)k:FY7zabYSYYdvfafjgTNTvS4z)kH2:Ff1kbMfL.\?J3*ZO!\kzR_;?"*)9utw D﻿pdxK;5MBNAV%[3YH7tOq[=\Zvz H&fDluY_0p·tO[(CEbVk7])9iu_y(sz5*WEd=Yky'WgtjMjaOL*ME!/nQ)93\4i{PLjb'RmO0*I6v!]LXM&\oM]Q.nv!WPakH﻿8y94]y2:q6J-mniLDRSa9EC*gaO!8IO\,[G'O!h%-U"E({gtltw"CabAgD?T:-3x)·gFcr.[ICy=um!8U5=R9S﻿pW3l&%qwcDS﻿nopIbk_W [6&O_MJA)U;zVQ)7xsaoZM/F}Me5h))oj"} %/Q;gEeS4_jC/d'fvbgxm&f4KtGP{;-j3'%RBAC)'Zw?X4DV74YK"[=48f=9!k,Lw

In [71]:
#x = train_data[:block_size]
#y = train_data[1:block_size + 1]

#for t in range(block_size):
#  context = x[:t + 1]
#  target = y[t]
#  print('when input is', context, 'target is', target)
import time
started_time = time.time()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
  if iter % eval_{iters}
    print(f"step: {iter}, loss: {losses}")
  xb, yb = get_batch('train')

  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
end_time = time.time()
print(f"time of training: {end_time - started_time}\nloss: {loss.item()}")


time of training: 8.57400369644165
loss: 2.815788984298706
